### Back to Instance Properties

Давайте попробуем использовать `WeakKeyDictionary` для хранения данных нашего экземпляра в нашем дескрипторе данных.

По сути, это то же самое, что мы делали раньше, но вместо использования стандартного словаря (который потенциально может вызывать утечки памяти) мы будем использовать `WeakKeyDictionary`.

Вспомним, что было раньше:

In [1]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[instance] = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)

Теперь мы собираемся реорганизовать это, чтобы использовать словарь слабых ключей:

In [2]:
import weakref

In [3]:
class IntegerValue:
    def __init__(self):
        self.values = weakref.WeakKeyDictionary()

    def __set__(self, instance, value):
        self.values[instance] = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)

И это все. Теперь в нашем словаре вместо сильных ссылок есть слабые, и словарь убирает за собой (удаляет «мертвые» записи), когда объект ссылки уничтожается GC.

In [4]:
class Point:
    x = IntegerValue()

In [5]:
p = Point()
print(hex(id(p)))

0x7fa760414400


In [6]:
p.x = 100.1

In [7]:
p.x

100

In [8]:
Point.x.values.keyrefs()

[<weakref at 0x7fa76041d048; to 'Point' at 0x7fa760414400>]

А если мы удалим `p`, тем самым удалив последнюю сильную ссылку на этот объект:

In [9]:
del p

In [10]:
Point.x.values.keyrefs()

[]

Итак, это почти идеальное общее решение:

1. Нам не нужно хранить данные в самих экземплярах (чтобы мы могли обрабатывать объекты, класс которых использует `__slots__`)
2. Мы защищены от утечек памяти

Но это работает только для **хешируемых** объектов.

Итак, теперь давайте попробуем решить эту проблему хешируемости.

Поскольку мы не можем использовать сам объект в качестве ключа в словаре (слабом или ином), мы могли бы попробовать использовать `id` объекта (который является целым числом) в качестве ключа в стандартном словаре:

In [11]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(id(instance))

Теперь мы можем использовать этот подход с нехешируемыми объектами:

In [12]:
class Point:
    x = IntegerValue()

    def __init__(self, x):
        self.x = x

    def __eq__(self, other):
        return isinstance(other, Point) and self.x == other.x

In [13]:
p = Point(10.1)

In [14]:
p.x

10

In [15]:
p.x = 20.2

In [16]:
p.x

20

In [17]:
id(p), Point.x.values

(140356851267288, {140356851267288: 20})

Теперь у нас больше нет утечки памяти:

In [18]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [19]:
p_id = id(p)

In [20]:
ref_count(p_id)

1

In [21]:
del p

In [22]:
ref_count(p_id)

-1

Но теперь у нас есть "мертвая" запись в нашем словаре - этот адрес памяти все еще присутствует как ключ. Теперь вы можете подумать, что это не так уж важно, но Python действительно повторно использует адреса памяти, поэтому мы можем столкнуться с потенциальными проблемами (где дескриптор данных будет иметь значение для свойства, уже установленного из предыдущего объекта), а также с тем фактом, что наш словарь загроможден этими мертвыми записями:

In [23]:
Point.x.values

{140356851267288: 20}

Поэтому нам нужен способ определить, был ли объект уничтожен.

Мы знаем, что слабые ссылки знают, когда объекты уничтожаются:

In [24]:
p = Point(10.1)
weak_p = weakref.ref(p)

In [25]:
print(hex(id(p)), weak_p)
# again note how I need to use print to avoid affecting the ref count

0x7fa76043c588 <weakref at 0x7fa760439318; to 'Point' at 0x7fa76043c588>


In [26]:
ref_count(id(p))

1

А если я уберу последнюю сильную ссылку на `p`:

In [27]:
del p

In [28]:
print(weak_p)

<weakref at 0x7fa760439318; dead>


Вы можете видеть, что слабая ссылка была уведомлена об этом изменении — на самом деле мы тоже можем это сделать, указав функцию **обратного вызова**, которую Python вызовет, как только слабая ссылка станет неактивной (т. е. объект будет уничтожен сборщиком мусора):

In [29]:
def obj_destroyed(obj):
    print(f'{obj} is being destroyed')

In [30]:
p = Point(10.1)
w = weakref.ref(p, obj_destroyed)

In [31]:
del p

<weakref at 0x7fa760439f48; dead> is being destroyed


Как видите, функция обратного вызова получает в качестве аргумента объект слабой ссылки.

Итак, мы можем использовать это в наших интересах в нашем дескрипторе данных, зарегистрировав обратный вызов, который мы можем использовать для удаления «мертвой» записи из нашего словаря значений.

Это означает, что нам также нужно сохранить слабую ссылку на объект — мы сделаем это в значении словаря `values` как часть кортежа, содержащего слабую ссылку на объект и соответствующее значение):

In [32]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object),
                                     int(value)
                                    )

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.values.get(id(instance))
            return value_tuple[1]  # return the associated value, not the weak ref

    def _remove_object(self, weak_ref):
        print(f'removing dead entry for {weak_ref}')
        # how do we find that weak reference?

Давайте просто убедимся, что наш обратный вызов выполняется так, как и ожидалось:

In [33]:
class Point:
    x = IntegerValue()

In [34]:
p1 = Point()
p2 = Point()

In [35]:
p1.x, p2.x = 10.1, 100.1

In [36]:
p1.x, p2.x

(10, 100)

Теперь давайте удалим эти объекты:

In [37]:
ref_count(id(p1)), ref_count(id(p2))

(1, 1)

In [38]:
del p1

removing dead entry for <weakref at 0x7fa760420cc8; dead>


In [39]:
del p2

removing dead entry for <weakref at 0x7fa760451098; dead>


Хорошо, теперь осталось только удалить соответствующую запись из словаря. Проблема в том, что на тот момент у нас нет самого объекта (и, следовательно, нет его идентификатора), поэтому мы не можем добраться до элемента словаря с помощью ключа — нам просто придется перебирать значения в словаре, пока не найдем значение, первым элементом которого является слабая ссылка, вызвавшая обратный вызов:

In [40]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object),
                                     int(value)
                                    )

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.values.get(id(instance))
            return value_tuple[1]  # return the associated value, not the weak ref

    def _remove_object(self, weak_ref):
        reverse_lookup = [key for key, value in self.values.items()
                         if value[0] is weak_ref]
        if reverse_lookup:
            # key found
            key = reverse_lookup[0]
            del self.values[key]

In [41]:
class Point:
    x = IntegerValue()

In [42]:
p = Point()

In [43]:
p.x = 10.1

In [44]:
p.x

10

In [45]:
Point.x.values

{140356851302352: (<weakref at 0x7fa760451db8; to 'Point' at 0x7fa760437fd0>,
  10)}

Теперь давайте удалим нашу (единственную) сильную ссылку на `p`:

In [46]:
ref_count(id(p))

1

In [47]:
del p

In [48]:
Point.x.values

{}

И как вы видите, наш словарь был очищен.

Есть еще одно предостережение: когда мы создаем слабые ссылки на объекты, объекты слабых ссылок фактически сохраняются в самом экземпляре, в свойстве с именем `__weakref__`:

In [49]:
class Person:
    pass

In [50]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

Обратите внимание на атрибут `__weakref__`. Технически это дескриптор данных:

In [51]:
hasattr(Person.__weakref__, '__get__'), hasattr(Person.__weakref__, '__set__')

(True, True)

И экземпляры, следовательно, будут обладать этим свойством:

In [52]:
p = Person()

In [53]:
hasattr(p, '__weakref__')

True

In [54]:
print(p.__weakref__)

None


Как вы видите, атрибут `__weakref__` существует, но в данный момент это `None`.

Теперь давайте создадим слабую ссылку на `p`:

In [55]:
w = weakref.ref(p)

И `__weakref__` больше не `None` (внутренне он реализован как двусвязный список всех слабых ссылок на этот объект, но это деталь реализации, и Python не предоставляет функциональность для итерации по слабым ссылкам самостоятельно)

In [56]:
p.__weakref__

<weakref at 0x7fa760451db8; to 'Person' at 0x7fa7603f2d68>

Проблема в том, что если мы используем слоты, то экземпляры больше не будут иметь этого атрибута!

In [57]:
class Person:
    __slots__ = 'name',

In [58]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__slots__': ('name',),
              'name': <member 'name' of 'Person' objects>,
              '__doc__': None})

Как видите, `__weakref__` больше не является атрибутом в нашем классе, и экземпляры его не имеют:

In [59]:
p = Person()

In [60]:
hasattr(p, '__weakref__')

False

Итак, проблема в том, что мы больше не можем создавать слабые ссылки на этот объект!!

In [61]:
try:
    weakref.ref(p)
except TypeError as ex:
    print(ex)

cannot create weak reference to 'Person' object


Чтобы включить слабые ссылки в объектах, использующих слоты, нам нужно указать `__weakref__` в качестве одного из слотов:

In [62]:
class Person:
    __slots__ = 'name', '__weakref__'

In [63]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__slots__': ('name', '__weakref__'),
              'name': <member 'name' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

Как видите, `__weakref__` вернулся и существует в наших экземплярах:

In [64]:
p = Person()

In [65]:
hasattr(p, '__weakref__')

True

Это значит, что мы снова можем создавать слабые ссылки на наш объект `Person`:

In [66]:
w = weakref.ref(p)

Итак, если мы хотим использовать дескрипторы данных, использующие слабые ссылки (используя наш собственный словарь или словарь слабых ключей) с классами, определяющими слоты, нам нужно убедиться, что мы добавляем `__weakref__` к слотам!

Давайте рассмотрим еще один пример, используя эту новейшую технику:

In [67]:
class ValidString:
    def __init__(self, min_length=0, max_length=255):
        self.data = {}
        self._min_length = min_length
        self._max_length = max_length

    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError('Value must be a string.')
        if len(value) < self._min_length:
            raise ValueError(
                f'Value should be at least {self._min_length} characters.'
            )
        if len(value) > self._max_length:
            raise ValueError(
                f'Value cannot exceed {self._max_length} characters.'
            )
        self.data[id(instance)] = (weakref.ref(instance, self._finalize_instance),
                                   value
                                  )

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.data.get(id(instance))
            return value_tuple[1]

    def _finalize_instance(self, weak_ref):
        reverse_lookup = [key for key, value in self.data.items()
                         if value[0] is weak_ref]
        if reverse_lookup:
            # key found
            key = reverse_lookup[0]
            del self.data[key]

Теперь мы можем использовать `ValidString` столько раз, сколько нам нужно:

In [68]:
class Person:
    __slots__ = '__weakref__',

    first_name = ValidString(1, 100)
    last_name = ValidString(1, 100)

    def __eq__(self, other):
        return (
            isinstance(other, Person) and
            self.first_name == other.first_name and
            self.last_name == other.last_name
        )

class BankAccount:
    __slots__ = '__weakref__',

    account_number = ValidString(5, 255)

    def __eq__(self, other):
        return (
            isinstance(other, BankAccount) and
            self.account_number == other.account_number
        )

In [69]:
p1 = Person()

In [70]:
try:
    p1.first_name = ''
except ValueError as ex:
    print(ex)

Value should be at least 1 characters.


In [71]:
p2 = Person()

In [72]:
p1.first_name, p1.last_name = 'Guido', 'van Rossum'
p2.first_name, p2.last_name = 'Raymond', 'Hettinger'

In [73]:
b1, b2 = BankAccount(), BankAccount()

In [74]:
b1.account_number, b2.account_number = 'Savings', 'Checking'

In [75]:
p1.first_name, p1.last_name

('Guido', 'van Rossum')

In [76]:
p2.first_name, p2.last_name

('Raymond', 'Hettinger')

In [77]:
b1.account_number, b2.account_number

('Savings', 'Checking')

Мы можем посмотреть словарь данных в каждом из экземпляров дескриптора данных:

In [78]:
Person.first_name.data

{140356851360776: (<weakref at 0x7fa76043e818; to 'Person' at 0x7fa760446408>,
  'Guido'),
 140356851360152: (<weakref at 0x7fa7400752c8; to 'Person' at 0x7fa760446198>,
  'Raymond')}

In [79]:
Person.last_name.data

{140356851360776: (<weakref at 0x7fa740075138; to 'Person' at 0x7fa760446408>,
  'van Rossum'),
 140356851360152: (<weakref at 0x7fa740075598; to 'Person' at 0x7fa760446198>,
  'Hettinger')}

In [80]:
BankAccount.account_number.data

{140356851360536: (<weakref at 0x7fa76043e868; to 'BankAccount' at 0x7fa760446318>,
  'Savings'),
 140356851361256: (<weakref at 0x7fa740075868; to 'BankAccount' at 0x7fa7604465e8>,
  'Checking')}

А если наши объекты являются мусоросборниками:

In [81]:
del p1
del p2
del b1
del b2

In [82]:
Person.first_name.data

{}

In [83]:
Person.last_name.data

{}

In [84]:
BankAccount.account_number.data

{}

мы видим, что наши словари тоже были очищены!

Итак, это был долгий путь, но теперь он позволяет нам обрабатывать классы, которые используют слоты и не являются хешируемыми.

В зависимости от ваших потребностей, вам может не понадобиться вся эта функциональность (например, ваши объекты могут быть гарантированно хешируемыми и поддерживать слабые ссылки, в этом случае вы можете использовать подход со словарем слабых ключей), или, может быть, ваш класс гарантированно не будет использовать слоты (или содержит `__dict__` в качестве одного из слотов), в этом случае вы можете просто использовать сам экземпляр для хранения (хотя имя для использования все еще остается нерешенным вопросом).

В следующем цикле лекций мы вернемся к использованию экземпляра для хранения вместо использования самого описания данных.

---